# Überblick über Basis-Tools: `gensim` und spezielle Tools

`gensim` ist eine Python-Bibliothek, die  primär für Topic Models entwickelt wurde, kann aber auch gut für Word Embeddings umgehen.

Mithilfe von PyLDAvis lassen sich Topic Modelle sehr intuitiv darstellen.

Im letzten Teil des Kurses werden wir einige Deep Learning Bibliotheken verwenden, die für *Sprachmodelle* wie BERT und Question Answering verwendet werden können.

## Topic Models

Das API ist von `gensim` ist etwas anders als bei `scikit-learn` und einige Aufgaben müssen *von Hand* erledigt werden. Wir können das Datenset mit den 20 Newsgroups von `scikit-learn` nutzen:

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups = fetch_20newsgroups()

Das Dictionary musst du bei `gensim` etwas aufwändiger aufgebauen, stark vereinfacht nutzt du Tokens, die durch den Tokenizier getrennt werden. Die *List Comprehensions* von Python machen deinen Code relativ kompakt:

In [ ]:
!pip install gensim

In [ ]:
from gensim.corpora import Dictionary
from gensim.utils import simple_tokenize

posts = [post for post in newsgroups['data']]
dict_voc = Dictionary([[word for word in simple_tokenize(post)] for post in posts]) 

Anschließend kannst du mithilfe des Dictionaries die Texte in sog. *Bag of Words*-Vektoren wandeln:

In [ ]:
bow = [dict_voc.doc2bow(simple_tokenize(post)) for post in posts]

LDA (*Latent Dirichlet Allocation*) ist eine der beliebtesten Methoden zur Berechnung von Topic Models. Leider benötigt das Verfahren durch sog. *Sampling* sehr viel Zeit. Im Gegensatz zu `scikit-learn` unterstüzt `gensim` dabei mehrere Prozessoren, wenn du einen entsprechenden Computer hast (Grafikkarten werden leider noch nicht genutzt). 

In [ ]:
from gensim.models.ldamulticore import LdaMulticore
lda = LdaMulticore(corpus=bow, id2word=dict_voc, chunksize=2000, iterations=400, num_topics=5, 
                   passes=20, eval_every=None, random_state=42)

Die Ausgabe der Modelle kannst du etwas einfacher erreichen als mit `scikit-learn`:

In [ ]:
lda.print_topics()

Die Ergebnisse sind nicht gut interpretierbar, was an der *schlechten Tokenisierung* liegt. Topic Models werden daher auch oft zur Qualitätssicherung eingesetzt - sowohl für die Daten als auch für die Verarbeitungspipeline.

## Visualisierung

Bestimmt interessierst du dich jetzt dafür, wie trennscharf die Topics sind. Dafür gibt es ein eigenes Paket `pyLDAvis`, das die LDA-Modelle von `scikit-learn` und `gensim` direkt darstellen kann. Evtl. musst du das Paket zuerst installieren:

In [ ]:
!pip install pyLDAvis==3.3.1

In [ ]:
import pyLDAvis.gensim_models

lda_display = pyLDAvis.gensim_models.prepare(lda, bow, dict_voc)
pyLDAvis.display(lda_display)

Wie du sehen kannst, überlappen sich die Topic-Modelle in dieser auf zwei Dimensionen reduzierten Darstellung deutlich - ein Zeichen für eine schlechte Modellierung

## Embeddings

Neben Topic Models kannst du `gensim` auch für *Word Embeddings* nutzen. Dabei wird jedem Wort ein Vektor zugeordnet, du benötigst dafür zwar keine vorklassifizierte Trainingsmenge, das Verfahren wird aber als *semi supervised* bezeichnet.

Wir haben als Korpus den Heise-Newsticker benutzt und daraus word2vec-Embeddings berechnet. Die Embeddings lassen sich einfach laden:

In [ ]:
import gensim

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.w2v || wget https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.w2v.gz && gunzip heise-articles-2020.w2v.gz")
    newsticker_w2v = 'heise-articles-2020.w2v'
else:
    newsticker_w2v = '../99_Common/heise-articles-2020.w2v'

In [ ]:
w2v = gensim.models.KeyedVectors.load_word2vec_format(newsticker_w2v, binary=False)

Du kannst nun semantisch ähnliche Wörter bestimmen:

In [ ]:
w2v.most_similar("apple")

Wenn du gerade nicht weißt, wie das Handy-Betriebssystem von *Google* heißt, sondern dir nur *iOS* von *Apple* einfällt, kann dir `word2vec` auch helfen.

In [ ]:
w2v.most_similar(positive=["google", "ios"], negative=["apple"])

Wir werden uns später noch eingehend mit Embeddings und deren spannenden Möglichkeiten beschäftigen. 